# Solve Single Crosswalk using QMDP and SARSOP

In [1]:
using POMDPs, StatsBase, POMDPToolbox, DeepRL, Parameters, GridInterpolations
using AutomotiveDrivingModels,AutoViz
using Reel 
using QMDP, SARSOP

In [2]:
using AutomotivePOMDPs

In [3]:
rng = MersenneTwister(1);

## Initialize Crosswalk environment

In [6]:
pomdp = SingleOCPOMDP(ΔT = 0.5, p_birth = 0.3)

06-Aug 09:25:38:WARNING:root:replacing docs for 'AutomotivePOMDPs.CrosswalkEnv :: Union{Tuple{AutomotivePOMDPs.CrosswalkParams}, Tuple{}}' in module 'AutomotivePOMDPs'.


AutomotivePOMDPs.SingleOCPOMDP(AutomotivePOMDPs.CrosswalkEnv(Roadway, AutomotiveDrivingModels.Lane(LaneTag(2, 1), AutomotiveDrivingModels.CurvePt[CurvePt({25.000, -10.000, 1.571}, 0.000, 0.000, NaN), CurvePt({25.000, 10.000, 1.571}, 20.000, 0.000, NaN)], 6.0, AutomotiveDrivingModels.SpeedLimit(-Inf, Inf), AutomotiveDrivingModels.LaneBoundary(:unknown, :unknown), AutomotiveDrivingModels.LaneBoundary(:unknown, :unknown), AutomotiveDrivingModels.LaneConnection[], AutomotiveDrivingModels.LaneConnection[]), AutomotiveDrivingModels.ConvexPolygon[ConvexPolygon: len 4 (max 4 pts)
	VecE2(15.000, -1.500)
	VecE2(15.000, -4.500)
	VecE2(21.500, -4.500)
	VecE2(21.500, -1.500)
], AutomotivePOMDPs.CrosswalkParams(2, 50.0, 3.0, 20.0, 6.0, 5.0, 37.0, 8.0, 100, 0.5, 2.0, 10.0)), VehicleDef(CAR, 4.000, 1.800), VehicleDef(PEDESTRIAN, 1.000, 1.000), 2.0, 1.0, 1.0, 5.0, -5.0, 37.0, 5.0, 0.5, 0.3, 0.3, false, 1.0, 1.0, 1.0, -1.0, 0.0, 1.0, 0.95)

In [7]:
policy = solve(RandomSolver(rng), pomdp)
up = updater(policy);

In [8]:
hr = HistoryRecorder(rng=rng, max_steps=200)
@time hist = simulate(hr, pomdp, policy, up);

  3.222652 seconds (1.54 M allocations: 73.924 MiB, 1.47% gc time)


In [9]:
duration, fps, render_hist = animate_hist(pomdp, hist, SceneOverlay[])
film = roll(render_hist, fps = fps, duration = duration)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmpHz9206", 0x0000000000000042, 2.0, nothing)

## Solve using QMDP

In [14]:
solver = QMDPSolver(max_iterations=100, tolerance=1e-3, verbose=true)

QMDP.QMDPSolver(100, 0.001, true)

In [15]:
qmdp_policy = solve(solver, pomdp)

[Iteration 1   ] residual:          1 | iteration runtime:   2485.151 ms, (      2.49 s total)
[Iteration 2   ] residual:       0.95 | iteration runtime:   2570.471 ms, (      5.06 s total)
[Iteration 3   ] residual:      0.903 | iteration runtime:   2674.629 ms, (      7.73 s total)
[Iteration 4   ] residual:      0.857 | iteration runtime:   2423.629 ms, (      10.2 s total)
[Iteration 5   ] residual:      0.811 | iteration runtime:   2424.142 ms, (      12.6 s total)
[Iteration 6   ] residual:      0.762 | iteration runtime:   2549.645 ms, (      15.1 s total)
[Iteration 7   ] residual:      0.707 | iteration runtime:   2928.046 ms, (      18.1 s total)
[Iteration 8   ] residual:      0.649 | iteration runtime:   2481.565 ms, (      20.5 s total)
[Iteration 9   ] residual:      0.539 | iteration runtime:   2532.580 ms, (      23.1 s total)
[Iteration 10  ] residual:      0.472 | iteration runtime:   3136.414 ms, (      26.2 s total)
[Iteration 11  ] residual:       0.42 | iteration 

POMDPToolbox.AlphaVectorPolicy{AutomotivePOMDPs.SingleOCPOMDP,AutomotivePOMDPs.SingleOCAction}(AutomotivePOMDPs.SingleOCPOMDP(AutomotivePOMDPs.CrosswalkEnv(Roadway, AutomotiveDrivingModels.Lane(LaneTag(2, 1), AutomotiveDrivingModels.CurvePt[CurvePt({25.000, -10.000, 1.571}, 0.000, 0.000, NaN), CurvePt({25.000, 10.000, 1.571}, 20.000, 0.000, NaN)], 6.0, AutomotiveDrivingModels.SpeedLimit(-Inf, Inf), AutomotiveDrivingModels.LaneBoundary(:unknown, :unknown), AutomotiveDrivingModels.LaneBoundary(:unknown, :unknown), AutomotiveDrivingModels.LaneConnection[], AutomotiveDrivingModels.LaneConnection[]), AutomotiveDrivingModels.ConvexPolygon[ConvexPolygon: len 4 (max 4 pts)
	VecE2(15.000, -1.500)
	VecE2(15.000, -4.500)
	VecE2(21.500, -4.500)
	VecE2(21.500, -1.500)
], AutomotivePOMDPs.CrosswalkParams(2, 50.0, 3.0, 20.0, 6.0, 5.0, 37.0, 8.0, 100, 0.5, 2.0, 10.0)), VehicleDef(CAR, 4.000, 1.800), VehicleDef(PEDESTRIAN, 1.000, 1.000), 2.0, 1.0, 1.0, 5.0, -5.0, 37.0, 5.0, 0.5, 0.3, 0.3, false, 1.0, 1

In [16]:
up = SingleOCUpdater(pomdp);

In [17]:
hr = HistoryRecorder(rng=rng)
@time hist = simulate(hr, pomdp, qmdp_policy, up);

 45.029022 seconds (658.04 M allocations: 17.745 GiB, 16.49% gc time)


In [18]:
duration, fps, render_hist = animate_hist(pomdp, hist)
film = roll(render_hist, fps = fps, duration = duration)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmpRckSFs", 0x0000000000000013, 2.0, nothing)

## QMDP Utility Fusion

In [ ]:
const DecBelief = Dict{Int64, SingleOCDistribution}
const DecState = Dict{Int64, SingleOCState}
const DecObs = Dict{Int64, SingleOCObs}

In [ ]:
pomdp = OCPOMDP()

In [ ]:
up = MixedUpdater(pomdp, pomdp.ΔT)

In [ ]:
hr = HistoryRecorder(rng=rng)
s0 = initial_state(pomdp, rng)
@time hist = simulate(hr, pomdp, qmdp_policy, up, d0, s0);

## Solve using SARSOP

In [9]:
solver = SARSOPSolver(randomization=true, precision = 0.001, timeout = 600.)


SARSOP.SARSOPSolver(Dict{AbstractString,Any}(Pair{AbstractString,Any}("randomization", ""),Pair{AbstractString,Any}("timeout", 600.0)))

In [10]:
sarsop_policy = solve(solver, pomdp,
               SARSOP.create_policy(solver, pomdp, "crosswalk.policy"), pomdp_file_name="crosswalk.pomdpx" )

Generating a pomdpx file: crosswalk.pomdpx

Loading the model ...
  input file   : crosswalk.pomdpx
  loading time : 148.62s 

SARSOP initializing ...
  initialization time : 178.54s

-------------------------------------------------------------------------------
 Time   |#Trial |#Backup |LBound    |UBound    |Precision  |#Alphas |#Beliefs  
-------------------------------------------------------------------------------
 178.544 0       0        12.1169    12.1191    0.00217179  4        1        
 208.314 4       19       12.1182    12.1191    0.000954115 17       11       
-------------------------------------------------------------------------------

SARSOP finishing ...
  target precision reached
  target precision  : 0.001000
  precision reached : 0.000954 

-------------------------------------------------------------------------------
 Time   |#Trial |#Backup |LBound    |UBound    |Precision  |#Alphas |#Beliefs  
-----------------------------------------------------------------

SARSOP.POMDPPolicy("crosswalk.policy", POMDPXFiles.POMDPAlphas([10.2753 9.80559 … 10.2753 10.2753; 10.3449 9.93102 … 10.3449 10.3449; … ; 19.9998 19.9998 … 19.9999 19.9999; 19.9998 19.9998 … 19.9999 19.9999], [3, 1, 3, 3, 3, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3]), AutomotivePOMDPs.SingleOCPOMDP(AutomotivePOMDPs.CrosswalkEnv(Roadway, AutomotiveDrivingModels.Lane(LaneTag(2, 1), AutomotiveDrivingModels.CurvePt[CurvePt({25.000, -10.000, 1.571}, 0.000, 0.000, NaN), CurvePt({25.000, 10.000, 1.571}, 20.000, 0.000, NaN)], 6.0, AutomotiveDrivingModels.SpeedLimit(-Inf, Inf), AutomotiveDrivingModels.LaneBoundary(:unknown, :unknown), AutomotiveDrivingModels.LaneBoundary(:unknown, :unknown), AutomotiveDrivingModels.LaneConnection[], AutomotiveDrivingModels.LaneConnection[]), AutomotiveDrivingModels.ConvexPolygon[ConvexPolygon: len 4 (max 4 pts)
	VecE2(15.000, -1.500)
	VecE2(15.000, -4.500)
	VecE2(21.500, -4.500)
	VecE2(21.500, -1.500)
], AutomotivePOMDPs.CrosswalkParams(2, 50.0, 3.0, 20.0, 6.0, 5.0, 

In [37]:
hr = HistoryRecorder(rng=rng)
@time hist = simulate(hr, pomdp, sarsop_policy, DiscreteUpdater(pomdp));

 11.688374 seconds (165.43 M allocations: 13.435 GiB, 16.18% gc time)


In [38]:
duration, fps, render_hist = animate_hist(pomdp, hist)
film = roll(render_hist, fps = fps, duration = duration)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmpoBsqbv", 0x0000000000000009, 2.0, nothing)